In [11]:
import pysilcam.process as process
import pysilcam.silcam_classify as classify
from pysilcam.background import backgrounder
from pysilcam.config import PySilcamSettings
import os
import imageio
import pandas as pd
import numpy as np
import logging

In [15]:
model_path = '/home/bjarne/nasgul/Miljoteknologi/SilCam/model/particle-classifier.tfl'
data_path = '/home/bjarne/raid/0_temp/silcam/indata/'
config_path = '/home/bjarne/workspace/pysilcam/pysilcam/config_example.ini'
logger = logging.getLogger(__name__ + '.silcam_process')
settings = PySilcamSettings(config_path)

In [16]:
# generators.... -_-
def acquire_gen(data_path):
    frame_paths = [data_path + o for o in os.listdir(data_path) if o.endswith(('.silc', '.bmp'))]
    for frame_path in frame_paths:
        if frame_path.endswith('.silc'):
            img = np.load(frame_path, allow_pickle=False)
        else:
            img = imageio.imread(frame_path)
        timestamp = pd.to_datetime(frame_path.split('/')[-1].split('.')[0][1:])
        yield timestamp, img
        
acquire = acquire_gen(data_path)

back_gen = backgrounder(10, acquire, bad_lighting_limit=None, real_time_stats=False)

In [17]:
# Load model
nnmodel, class_labels = classify.load_model(model_path=model_path)


INFO:tensorflow:Restoring parameters from /home/bjarne/nasgul/Miljoteknologi/SilCam/model/particle-classifier.tfl


INFO:tensorflow:Restoring parameters from /home/bjarne/nasgul/Miljoteknologi/SilCam/model/particle-classifier.tfl


In [10]:
for i, (timestamp, imc, imraw) in enumerate(back_gen):
    
    image = (i, timestamp, imc)
    stats_all = process.processImage(nnmodel, class_labels, image, settings)

  Image 0 processed in 2.14 sec (0.5 Hz).
    major_axis_length  minor_axis_length  equivalent_diameter  solidity  \
0           42.113996          38.353357            40.180440  0.980665   
1           24.680025          21.366632            22.959120  0.981043   
2            7.023817           6.131904             6.579525  0.944444   
3           60.852419          59.278819            60.037831  0.971517   
4            4.992162           4.350410             4.652426  1.000000   
5          285.414221         244.364265           263.905671  0.993769   
6          183.092833         131.160295           148.787822  0.917617   
7           11.000680          10.526972            10.704745  0.937500   
8           61.011717          58.662591            59.814740  0.982174   
9           11.375719           9.744116            10.464158  0.966292   
10          26.267731          25.128452            25.631847  0.975425   
11          68.640503          65.218425            66.851